# Semantic ID Tutorial: Text Clustering with AG News

Welcome to this tutorial on **Semantic ID**! 

In this notebook, we will demonstrate how to generate semantic identifiers for a textual dataset (AG News). We will cover:

1.  **RQ-KMeans**: Hierarchical clustering on CPU, GPU, and MPS.
2.  **Uniqueness**: Handling ID collisions automatically.
3.  **RQ-VAE**: Using neural networks for quantization.

## Prerequisites

First, let's install the required libraries.

In [1]:
# !pip install semantic-id datasets sentence-transformers pandas

## Imports and Setup

In [2]:
import torch
import numpy as np
import pandas as pd
from datasets import load_dataset
from sentence_transformers import SentenceTransformer

# Semantic ID imports
from semantic_id.algorithms.rq_kmeans import RQKMeans
from semantic_id.algorithms.rq_vae import RQVAE
from semantic_id.engine import SemanticIdEngine
from semantic_id.uniqueness.resolver import UniqueIdResolver
from semantic_id.uniqueness.stores import SQLiteCollisionStore

# Set random seed for reproducibility
np.random.seed(42)
torch.manual_seed(42)

/Users/mikhail/VSCodeProjects/semantic-id/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 1. Load Data and Generate Embeddings

We will use the **AG News** dataset, which contains news articles categorized into World, Sports, Business, and Sci/Tech. This is a perfect dataset to see if our IDs capture semantic meaning (e.g., all Sports articles should have similar IDs).

In [3]:
# Load dataset
print("Loading AG News dataset...")
dataset = load_dataset("ag_news", split="train")

# Subsample for this tutorial (20,000 samples)
# This keeps the execution fast while providing enough data for clustering.
subset_size = 20000
dataset = dataset.shuffle(seed=42).select(range(subset_size))

texts = dataset["text"]
labels = dataset["label"]
label_names = dataset.features["label"].names

print(f"Loaded {len(texts)} samples.")
print(f"Example category: {label_names[labels[0]]}")
print(f"Example text: {texts[0][:100]}...")

Loading AG News dataset...
Loaded 20000 samples.
Example category: World
Example text: Bangladesh paralysed by strikes Opposition activists have brought many towns and cities in Banglades...


Now we generate vector embeddings for our text. We use `all-MiniLM-L6-v2` from SentenceTransformers, which is fast and efficient.

In [4]:
model_name = 'all-MiniLM-L6-v2'
embedding_model = SentenceTransformer(model_name)

print(f"Generating embeddings using {model_name}...")
embeddings = embedding_model.encode(texts, show_progress_bar=True)

print(f"Embeddings shape: {embeddings.shape}")

Loading weights: 100%|██████████| 103/103 [00:00<00:00, 1854.42it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Generating embeddings using all-MiniLM-L6-v2...


Batches: 100%|██████████| 625/625 [00:20<00:00, 30.86it/s]

Embeddings shape: (20000, 384)


## 2. RQ-KMeans (Residual Quantization K-Means)

This algorithm hierarchically clusters the data. 
- **Level 1**: Coarse clustering (e.g., broad topics)
- **Level 2**: Fine clustering (sub-topics)
- **Level 3+**: Even finer detail

We will use **4 levels** with **10 clusters** each. This gives us a theoretical capacity of $10^4 = 10,000$ unique buckets. Since we have 20,000 items, we *will* have collisions, which we'll handle later.

In [5]:
# Detect available device
if torch.cuda.is_available():
    device = "cuda"
    print("Using GPU (CUDA) 🚀")
elif torch.backends.mps.is_available():
    device = "mps"
    print("Using Mac MPS 🍎")
else:
    device = "cpu"
    print("Using CPU 🐢")

# Initialize RQ-KMeans
rq_kmeans = RQKMeans(
    n_levels=4, 
    n_clusters=10, 
    random_state=42,
    verbose=True
)

# Fit the model
print("Training RQ-KMeans...")
rq_kmeans.fit(embeddings, device=device)

Using Mac MPS 🍎
Training RQ-KMeans...
Training level 1/4 (K=10) on mps...
Training level 2/4 (K=10) on mps...
Training level 3/4 (K=10) on mps...
Training level 4/4 (K=10) on mps...


### Generate Semantic IDs
Now let's see what the IDs look like!

In [6]:
codes = rq_kmeans.encode(embeddings, device=device)
sids = rq_kmeans.semantic_id(codes)

# Create a DataFrame to analyze
df = pd.DataFrame({
    "text": texts,
    "label": [label_names[l] for l in labels],
    "semantic_id": sids
})

print(df.head(10))

                                                text     label semantic_id
0  Bangladesh paralysed by strikes Opposition act...     World     7-5-8-3
1  Desiring Stability Redskins coach Joe Gibbs ex...    Sports     1-1-2-3
2  Will Putin #39;s Power Play Make Russia Safer?...     World     2-0-0-8
3  U2 pitches for Apple New iTunes ads airing dur...  Sci/Tech     9-3-2-0
4  S African TV in beheading blunder Public broad...     World     7-6-4-3
5  A Cosmic Storm: When Galaxy Clusters Collide A...  Sci/Tech     3-9-6-4
6  West sets deadline for Iran to freeze uranium ...     World     4-1-6-0
7  Computer Assoc. Cuts 800 Jobs Worldwide (AP) A...  Sci/Tech     6-4-8-2
8  CA Opens Utility Pricing for Mainframes Keepin...  Sci/Tech     6-1-5-2
9  Economy builds steam in KC Fed district The ec...  Business     5-1-5-9


### Inspect Clustering Quality
Let's sort by `semantic_id` to see if similar articles are grouped together.

In [9]:
df_sorted = df.sort_values("semantic_id")
print(df_sorted[["semantic_id", "label", "text"]].head(20))

      semantic_id     label                                               text
437       0-0-0-0  Sci/Tech  Secret Service Busts Cyber Gangs The US Secret...
2321      0-0-0-0  Sci/Tech  ISA Server 2000 and Proxy Server 2.0 Internet ...
2552      0-0-0-0  Sci/Tech  Group Seeks Ways to Prosecute Cybercrime Gover...
2910      0-0-0-0  Sci/Tech  News: Terrorists grow fat on email scams Organ...
3540      0-0-0-0     World  Poison porn pics show up online The first imag...
4243      0-0-0-0  Sci/Tech  Possible security breach seen at AOL America O...
5428      0-0-0-0  Sci/Tech  Dozens of Internet Crime Suspects Nabbed A sum...
5889      0-0-0-0  Sci/Tech  'Phishing' scam targets NatWest NatWest bank s...
6059      0-0-0-0  Sci/Tech  How Not to Get  #39;Phished #39; With phishing...
6173      0-0-0-0  Sci/Tech  Banks Warned Against On-Line Fraud Dangers (Re...
6914      0-0-0-0  Sci/Tech  First US Spam Conviction A man was convicted i...
6915      0-0-0-0  Sci/Tech  Notice: IE  #39;worm #3

## 3. Handling Uniqueness (The Engine)

As predicted, we likely have duplicates (multiple items with the same ID like `3-9-1-0`). The `SemanticIdEngine` solves this by appending a counter.

Example: 
- Item 1 -> `3-9-1-0`
- Item 2 -> `3-9-1-0-1`
- Item 3 -> `3-9-1-0-2`

In [10]:
# 1. Setup Collision Store (SQLite for persistence)
# This saves the counters so if you restart the script, it remembers the next available suffix.
store = SQLiteCollisionStore("ag_news_collisions.db")

# 2. Setup Resolver
resolver = UniqueIdResolver(store=store)

# 3. Create Engine
engine = SemanticIdEngine(encoder=rq_kmeans, unique_resolver=resolver)

# 4. Generate Unique IDs
print("Generating unique IDs...")
unique_ids = engine.unique_ids(embeddings)

df["unique_id"] = unique_ids

# Show examples where collisions occurred (ID length > standard 7 chars)
collisions = df[df["unique_id"].str.len() > 7]
print(f"Total items with collision handling: {len(collisions)}")
print(collisions[["text", "unique_id"]].head())

Generating unique IDs...
Total items with collision handling: 14801
                                                  text  unique_id
64   Moss officially out for Green Bay EDEN PRAIRIE...  1-6-2-0-1
78   One last mile to go before they sweep Pedro Ma...  9-9-6-6-1
89   Over 80 hurt on first day of Bangladesh strike...  7-5-4-2-1
117  St. Louis Rams Release Punter Sean Landeta (AP...  1-6-2-0-2
138  Having the final say FOXBOROUGH -- The Patriot...  1-0-6-7-1


## 4. RQ-VAE (Neural Network Approach)

For complex distributions, **RQ-VAE** (Residual Quantization Variational Autoencoder) can learn better representations than K-Means. It uses a neural network to project data into codebooks.

In [12]:
print("Initializing RQ-VAE...")

rq_vae = RQVAE(
    in_dim=embeddings.shape[1], # 384
    num_emb_list=[16, 16, 16, 16], # 4 levels
    e_dim=32,                   # Codebook dimension
    layers=[128, 64],           # Encoder/Decoder layers
    device=device,
    batch_size=256,
    epochs=5,
    verbose=1,
    kmeans_init=True
)

print("Training RQ-VAE (5 epochs)...")
# Note: In a real scenario, train for more epochs!
rq_vae.fit(embeddings)

# Generate IDs
vae_codes = rq_vae.encode(embeddings, device=device)
vae_ids = rq_vae.semantic_id(vae_codes)

print(f"Sample VAE ID: {vae_ids[0]}")

Initializing RQ-VAE...
Training RQ-VAE (5 epochs)...
Epoch 1/5 - Loss: 0.0030 (Recon: 0.0025) - L0: 100.0% (Perp: 7.7) | L1: 87.5% (Perp: 11.9) | L2: 100.0% (Perp: 13.2) | L3: 87.5% (Perp: 10.8)
Epoch 2/5 - Loss: 0.0028 (Recon: 0.0024) - L0: 100.0% (Perp: 9.3) | L1: 87.5% (Perp: 12.6) | L2: 100.0% (Perp: 14.2) | L3: 81.2% (Perp: 11.1)
Epoch 3/5 - Loss: 0.0027 (Recon: 0.0024) - L0: 93.8% (Perp: 9.6) | L1: 87.5% (Perp: 13.1) | L2: 100.0% (Perp: 14.7) | L3: 81.2% (Perp: 11.7)
Epoch 4/5 - Loss: 0.0026 (Recon: 0.0023) - L0: 93.8% (Perp: 9.8) | L1: 87.5% (Perp: 13.5) | L2: 100.0% (Perp: 14.6) | L3: 87.5% (Perp: 12.3)
Epoch 5/5 - Loss: 0.0026 (Recon: 0.0023) - L0: 93.8% (Perp: 10.7) | L1: 93.8% (Perp: 13.7) | L2: 100.0% (Perp: 15.0) | L3: 93.8% (Perp: 12.8)
Sample VAE ID: 6-11-3-11


## Conclusion

You have successfully:
1.  Generated embeddings for text data.
2.  Clustered them into **Semantic IDs** using RQ-KMeans.
3.  Ensured every ID is unique using the **SemanticIdEngine**.
4.  Experimented with **RQ-VAE** for neural quantization.

Happy clustering! 🌟